# Bitbubble

Simplified ABM model that links the **"How to grow a bubble: A model of myopic adapting agents"** with the **"Leader–follower model for agent based simulation of social collective behavior during egress"** in the bitcoin bubble model space

In [1]:
import random
import numpy as np
import pandas as pd
import networkx as nx
import matplotlib as mpl
from mesa import Agent, Model
import matplotlib.pyplot as plt
from mesa.time import RandomActivation
%matplotlib inline
#print(mpl.style.available)
mpl.style.use('ggplot')
plt.xkcd()
np.random.seed(seed=42)

In [363]:
class MarketMaker(Model):
    """A model with some number of investors."""
    def __init__(self, N, market_depth, alpha=0.95, k=4, seed=None):
        super().__init__(seed)
        self.schedule = RandomActivation(self)
        self.N = N
        self.market_depth = market_depth
        self.alpha = alpha
        self.t = 0
        self.current_price = 1
        self.list_price = [self.current_price]
        self.list_return = [1]
        self.list_sp_return = [1]
        self.list_sigma_return = [1]
        self.list_news = []
        self.current_orders = []
        self.G = self.create_network(k)
        # Create agents
        for i in self.G.nodes():
            a = Investor(i, self)
            self.G.add_node(i)
            self.schedule.add(a)
            
    def step(self):
        '''Advance the model by one step.'''
        self.t = self.t + 1
        self.current_orders = list([])
        "Agents: move!"
        self.schedule.step()
        "After the orders, recalc the price of the asset"
        r = self.get_return()
        self.list_return.append(r)
        self.current_price = self.calc_price(r)
        self.list_price.append(self.current_price)
        self.list_sp_return.append(self.get_special_r(self.t))
        self.list_sigma_return.append(self.get_sigma_r(self.t))
        "Update wealth of agents with the new price"
        for a in self.schedule.agents:
            a.update_wealth(self.current_price)
        """TODO add end conditions (basic, upper limit, lower limit)"""
        if len([i for i, e in enumerate(self.current_orders) if e != 0]) > 0:
            print("orders for t", self.t, self.current_orders)
        
    def create_network(self, num_edges):
        '''Creates a Barabasi-Albert network with the number of agents and edges.'''
        return nx.generators.random_graphs.barabasi_albert_graph(self.N, num_edges)
    
    def newspaper(self, t):
        available_news = len(self.list_news)
        if available_news <= t:
            for i in range(0, t - (available_news - 1)):
                self.list_news.append(np.random.normal())
        return self.list_news[t]
    
    def get_price(self, t):
        price = self.list_price[0]
        if t > 0 and t < len(self.list_price):
            price = self.list_price[t]
        return price
    
    def add_order(self, unique_id, s=0, v=0.0):
        self.current_orders.append(s*v)
        
    def get_return(self):
        total = sum(self.current_orders)
        return total / (self.market_depth * self.N)
    
    def calc_price(self, r):
        price = np.exp(np.log(self.current_price) + r)
        return price
    
    def get_special_r(self, t):
        sp_r = self.list_sp_return[0]
        if t > 0 and t <= len(self.list_sp_return):
            sp_r = self.alpha * self.list_sp_return[t-1] + (1-self.alpha) * self.list_return[t-1]
        return sp_r
    
    def get_sigma_r(self, t):
        sigma_r = self.list_sigma_return[0]
        if t > 0 and t <= len(self.list_sigma_return):
            sigma_r = np.sqrt(
                self.alpha * np.square(self.list_sigma_return[t-1])
                + (1-self.alpha) * np.square(self.list_return[t-1] - self.list_sp_return[t-1])
            )
        return sigma_r

In [447]:
class Investor(Agent):
    """An investor with fixed initial cash and coins."""
    def __init__(self, unique_id, model, C1=1, C2=1, C3=1
                 , Omega=2, g=0.02, cash=1.0, coins=1.0):
        super().__init__(unique_id, model)
        self.cash = cash
        self.coins = coins
        self.Omega = Omega
        self.g = g
        #TODO: how to generate closed interval random numbers?
        self.c1 = np.random.uniform(0.0, C1)
        self.c2 = np.random.uniform(0.0, C1)
        self.c3 = np.random.uniform(0.0, C3)
        self.w_min = np.random.uniform(0.0, Omega)
        
        self.current_action = 0
        self.current_volume = 0
        self.list_opinions = [0]
        self.list_actions = [0]
        self.list_volumes = [0]
        self.list_cash = [self.cash]
        self.list_coins = [self.coins]
        self.dict_k = self.init_neighbors_k()
        self.private_info = [self.collect_private_info()]
        
    def step(self):
        time = self.model.t
        E = self.collect_private_info()
        self.private_info.append(E)
        opinion = self.get_market_opinion(E)
        self.list_opinions.append(opinion)
        self.current_action, self.current_volume = self.get_actions(opinion, time)
        self.list_actions.append(self.current_action)
        self.list_volumes.append(self.current_volume)
        self.model.add_order(self.unique_id, self.current_action, self.current_volume)
        
    def collect_private_info(self):
        return np.random.normal()
    
    def get_market_opinion(self, E):
        t = self.model.t
        opinion = (self.c1) * sum( [(self.get_neighbor_factor(neighbor, t-1) 
                           * self.get_neighbor_action(neighbor, t)) 
                          for neighbor in self.get_neighbors()] ) 
        + (self.c2 * self.get_news_factor(t-1) * self.model.newspaper(t)) 
        + (self.c3 * E)
        
        return opinion
        
    def get_neighbors(self):
        return [t[1] for t in self.model.G.edges(self.unique_id)]
    
    def init_neighbors_k(self):
        res = {}
        for n in self.get_neighbors():
            res[n] = [1]
            
        return res
        
    def get_neighbor_factor(self, neighbor_id, time):
        alpha = self.model.alpha
        k = self.dict_k[neighbor_id][0]
        prev_k_list = self.dict_k[neighbor_id]
        prev_k = prev_k_list[0]
        prev_n_action = self.get_neighbor_action(neighbor_id, 0)
        r = self.model.list_return[time]
        sigma_r = self.model.list_sigma_return[time]
        if time > 0 and time <= len(self.dict_k[neighbor_id]):
            prev_k = prev_k_list[time - 1]
            prev_n_action = self.get_neighbor_action(neighbor_id, time - 1)
            k = (alpha * prev_k) + ((1-alpha) * prev_n_action * r/sigma_r)
            prev_k_list.append(k)
            
        if time > 0 and r != 0 and sigma_r>0 and prev_n_action !=0:
            print(self.unique_id, time, "prev_n_action", prev_n_action
                  , "r", r, "sigma_r", sigma_r, "k", k)        

        return k
    
    def get_neighbor_action(self, neighbor_id, time):
        nb_action = random.choice([-1,0,1])
        for a in self.model.schedule.agents:
            if a.unique_id == neighbor_id:
                if time > 0 and time < len(a.list_actions):
                    nb_action = a.list_actions[time-1]
                    
        return nb_action
    
    def get_news_factor(self, time):
        return np.random.uniform(-1,1)
    
    def get_actions(self, opinion, time):
        s = 0
        v = 0
        if opinion >= self.w_min:
            s = 1
            v = self.g * self.cash / self.model.get_price(time-1)
        elif opinion <= self.w_min * (-1):
            s = -1
            v = self.g * self.coins
        return s, v
        
    def update_wealth(self, price):
        self.cash = self.cash - self.current_action * self.current_volume * price
        self.list_cash.append(self.cash)
        self.coins = self.coins + self.current_action * self.current_volume
        self.list_coins.append(self.coins)
        

In [450]:
bitmarket = MarketMaker(10, 0.25, seed=42)
for i in range(50):
    bitmarket.step()

orders for t 1 [0.02, 0, 0, 0, 0, 0, 0, 0, 0, 0]
1 1 prev_n_action 1 r 0.008 sigma_r 0.9746794344808963 k 0.950410391340834
1 1 prev_n_action 1 r 0.008 sigma_r 0.9746794344808963 k 0.950410391340834
9 1 prev_n_action -1 r 0.008 sigma_r 0.9746794344808963 k 0.9495896086591659
9 1 prev_n_action 1 r 0.008 sigma_r 0.9746794344808963 k 0.950410391340834
0 1 prev_n_action 1 r 0.008 sigma_r 0.9746794344808963 k 0.950410391340834
0 1 prev_n_action 1 r 0.008 sigma_r 0.9746794344808963 k 0.950410391340834
6 1 prev_n_action -1 r 0.008 sigma_r 0.9746794344808963 k 0.9495896086591659
6 1 prev_n_action -1 r 0.008 sigma_r 0.9746794344808963 k 0.9495896086591659
6 1 prev_n_action 1 r 0.008 sigma_r 0.9746794344808963 k 0.950410391340834
6 1 prev_n_action 1 r 0.008 sigma_r 0.9746794344808963 k 0.950410391340834
8 1 prev_n_action 1 r 0.008 sigma_r 0.9746794344808963 k 0.950410391340834
8 1 prev_n_action 1 r 0.008 sigma_r 0.9746794344808963 k 0.950410391340834
8 1 prev_n_action -1 r 0.008 sigma_r 0.974679